
# Atividade Named-Entity Recognition (NER)

### Desafio

Crie a rotina para processamento dos textos, preparando os documentos para respeitar a estrutura proposta para rodar uma rede neural capaz de realizar a atividade de reconhecimento de entidades.

Lembre que o resultado deverá ter três colunas (identificador, palavra e tag) como exemplificado na tabela abaixo:

| identificador | palavra | tag |
|:--:|:--:|:--:|
|102|tronco|O|
|102|da|O|
|102|coronaria|vaso|
|102|esquerda|vaso|
|102|com|O|
|102|trajeto|O|

Você deverá considerar as seguintes classes e termos por classe:

-   **vaso**:
	- descendente anterior;
	- coronaria direita;
	- coronaria esquerda;
	- circunflexa;
	- primeiro ramo marginal;
	- segundo ramo marginal;
	- terceiro ramo marginal;
	- primeiro ramo diagonal;
	- segundo ramo diagonal;
	- terceiro ramo diagonal;
	- ventricular posterior;
	- arteria diagonalis;
	- descendente posteiror.
-   **trajeto**:
	- trajeto intramiocardico;
	- origem;
	- retroaortico;
	- interaortopulmonar;
	- valsalva;
	- seio;
	- sinotubular.
-   **placa**:
	- placa;
	- placas;
	- ateromatose.
-   **composição**:
	- calcificada;
	- calcificadas;
	- densamente calcificada;
	- densamente calcificadas;
	- densa;
	- densamente calcificado;
	- parcialmente calcificada;
	- predominantemente calcificada;
	- predominantemente calcificadas;
	- mista;
	- mistas;
	- predominantemente nao calcificada;
	- predominantemente não calcificada;
	- parcialmente não calcificada;
	- principalmente calcificada;
	- principalmente não calcificada;
	- principalmente nao calcificada;
	- predominio calcificado;
	- predominio nao calcificado;
	- não calcificada.
-   **grau**:
	- sem redução luminal;
	- discreta;
	- irregularidades parietais;
	- irregularidade parietal;
	- menor que 50%;
	- irregularidades luminais;
	- irregularidade luminal;
	- irregularidade;
	- irregularidades;
	- proxima de 50%;
	- proximo de 50%;
	- entre 50 e 70%;
	- 50%;
	- acima de 50%;
	- maior que 50%;
	- ao redor de 50%;
	- cerca de 50%;
	- em torno de 50%;
	- acima de 70%;
	- 70%;
	- cerca de 70%;
	- ao redor de 70%;
	- em torno de 70%;
	- moderada;
	- moderada reducao luminal;
	- reducao luminal moderada;
	- estenose moderada;
	- grau pelo menos moderado;
	- grau moderado;
	- acentuada;
	- suboclusao;
	- reducao luminal critica;
	- estenose critica;
	- oclusao;
	- ocluido;
	- ocluida.
-   **modificador V**:
	- modificador V;
	- vulnerabilidade;
	- remodelamento positivo;
	- baixa atenuação;
	- napking ring;
	- anel de guardanapo;
	- spot calcifications;
	- remodelamento arterial positivo.
-   **stent**:
	- stent.
-   **redução stent**:
	- hiperplasia neointimal;
	- neointimal;
	- proliferação neointimal.
-   **enxerto**:
	- enxerto.

In [ ]:
# Resolução
import numpy as np
import pandas as pd
import re

pd.set_option('display.max_rows', None)

df = pd.read_excel('https://github.com/pgiaeinstein/ner-aula/raw/master/data_coronaria.xlsx')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
reg_vaso = re.compile(r"(descendente anterior|descendente posterior|coronaria (esquerda|direita)|circunflexa|ventricular posterior|arteria diagonalis|((primeiro|segundo|terceiro|quarto)(\se\s|\s))+(ramos?)?\s?(diagon(al|ais)?|margin(al|ais)?)?)")
reg_trajeto = re.compile(r"(trajeto intramiocardico|retroaórtico|interaortopulmonar|valssalva|sinotubular|seio)")
reg_placa = re.compile(r"(placas?|ateromatose)")
reg_desc_placa = re.compile(r"((predominantemente nao |nao |parcialmente |parcialmente nao |predominantemente |principalmente |principalmente nao |densamente )?calcificadas?|mistas?|predominio calcificado|predominio nao calcificado|densamente calcificado|densa)")
reg_oclusao = re.compile(r"(oclusao|suboclusao|acentuada|\b70%?\b|\b50%?\b|moderada\s?(reducao)?\s?(luminal)?|discreta\b|(irregularidades?\s?(pariet(ais|al)|lumin(al|ais))?)|menor que 50%?|maior que 50%?|entre 50%? e 70%?|acima de 50%?|ao redor de 50%?|cerca de 50%?|em torno de 50%?|acima de 70%?|cerca de 70%?|ao redor de 70%?|em torno de 70%?|reducao luminal critica|estenose critica|ocluid(o|a)|reducao luminal moderada|estenose moderada|grau pelo menos moderado|grau moderado|proxima de 50%?|proximo de 50%?)")
reg_modificador = re.compile(r"(modificador v|vulnerabilidade|remodelamento positivo|baixa atenuacao|napking ring|anel de guardanapo|spot calcifications|baixa atenuacao|remodelamento arterial positivo)")
reg_stent = re.compile(r"(stent)")
reg_red_stent = re.compile(r"(proliferacao neointimal|hiperplasia neointimal|hiperplasia|neointimal)")
reg_enxerto = re.compile(r"(enxerto)")

In [ ]:
lista_documentos = df['texto'].tolist()

In [ ]:
def encontra_tags(texto):
    
    lista_retorno = list()
    
    for item in reg_vaso.finditer(texto):
        lista_retorno.append({
            'classe'  : 'vaso',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
    
    for item in reg_trajeto.finditer(texto):
        lista_retorno.append({
            'classe'  : 'trajeto',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_placa.finditer(texto):
        lista_retorno.append({
            'classe'  : 'placa',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_desc_placa.finditer(texto):
        lista_retorno.append({
            'classe'  : 'desc_placa',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_oclusao.finditer(texto):
        lista_retorno.append({
            'classe'  : 'oclusao',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_modificador.finditer(texto):
        lista_retorno.append({
            'classe'  : 'modificador',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_stent.finditer(texto):
        lista_retorno.append({
            'classe'  : 'stent',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_red_stent.finditer(texto):
        lista_retorno.append({
            'classe'  : 'red_stent',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_enxerto.finditer(texto):
        lista_retorno.append({
            'classe'  : 'enxerto',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })

    return sorted(lista_retorno, key = lambda x : x['inicial'])

In [ ]:
def parse_texto(texto, lista_tags, indice):
    
    lista_retorno = list()
    lista_tmp = list()
    
    tamanho_texto = len(texto)
    tamanho_lista = len(lista_tags)
    
    ultima_pos = 0
    
    for index, achado in enumerate(lista_tags):
        
        if achado['inicial'] != 0 and index == 0:
            lista_tmp.append((texto[0:achado['inicial']], 'O'))
            ultima_pos = achado['inicial']

        if ultima_pos != achado['inicial']:
            lista_tmp.append((texto[ultima_pos:achado['inicial']], 'O'))
            
        lista_tmp.append((texto[achado['inicial']:achado['final']], achado['classe']))
        ultima_pos = achado['final']
            
        if index + 1 == tamanho_lista:
            lista_tmp.append((texto[ultima_pos:], 'O'))
        
    for item in lista_tmp:
        tag_atual = item[1]
        for palavra in item[0].split(' '):
            if palavra == '':
                continue
            lista_retorno.append({
                'identificador' : indice,
                'palavra'       : palavra,
                'tag'           : tag_atual
            }) 
    
    return lista_retorno
        

In [ ]:
index = 1
lista_resultante = list()

for documento in lista_documentos:
    lista_tags_documento = encontra_tags(documento)
    lista_resultante.extend(parse_texto(documento, lista_tags_documento, index))
    index += 1

In [ ]:
df_para_nn = pd.DataFrame(lista_resultante)

In [ ]:
df_para_nn.head(200)

### Modelos

In [ ]:
import pandas as pd
import numpy as np

# data = pd.read_excel("new_output.xlsx")

In [ ]:
data = df_para_nn

In [ ]:
# data = data.fillna(method="ffill")

In [ ]:
data.tail(10)

In [ ]:
f'Total de documentos na base: {len(data.identificador.unique())}'

In [ ]:
words = list(set(data["palavra"].values))
words.append("ENDPAD")
n_words = len(words)
n_words

In [ ]:
tags = list(set(data["tag"].values))
n_tags = len(tags)
n_tags

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["palavra"].values.tolist(),
                                                     s["tag"].values.tolist())]
        self.grouped = self.data.groupby("identificador").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["identificador".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(data)

In [ ]:
sent = getter.get_next()

In [ ]:
sentences = getter.sentences

In [ ]:
max_len = 300
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [ ]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)

In [ ]:
word2idx['ENDPAD']

In [ ]:
word2idx

In [ ]:
X[0]

In [ ]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]

In [ ]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [ ]:
lista_documentos[1]

In [ ]:
y[1]

In [ ]:
tag2idx

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, random_state=6)

### BI LSTM

In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [ ]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=32, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)

In [ ]:
model = Model(input, out)

In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=15, validation_split=0.1, verbose=1)

In [ ]:
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
test_pred = model.predict(X_te, verbose=1)

In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("ENDPAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [ ]:
print("ACC      : {:.4%}".format(accuracy_score(test_labels, pred_labels)))
print("PRECISION: {:.4%}".format(precision_score(test_labels, pred_labels)))
print("RECALL   : {:.4%}".format(recall_score(test_labels, pred_labels)))
print("F1       : {:.4%}".format(f1_score(test_labels, pred_labels)))

In [ ]:
print(classification_report(test_labels, pred_labels))

### BI-LSTM-CRF

In [ ]:
import keras

In [ ]:
keras.__version__

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [ ]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=32, input_length=max_len, mask_zero=True)(input)
model = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))(model)
# model = TimeDistributed(Dense(64, activation="relu"))(model)
crf = CRF(n_tags)
out = crf(model)

In [ ]:
model = Model(input, out)

In [ ]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=15, validation_split=0.1, verbose=1)

In [ ]:
hist = pd.DataFrame(history.history)

In [ ]:
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
test_pred = model.predict(X_te, verbose=1)

In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("ENDPAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [ ]:
print("ACC      : {:.4%}".format(accuracy_score(test_labels, pred_labels)))
print("PRECISION: {:.4%}".format(precision_score(test_labels, pred_labels)))
print("RECALL   : {:.4%}".format(recall_score(test_labels, pred_labels)))
print("F1       : {:.4%}".format(f1_score(test_labels, pred_labels)))

In [ ]:
print(classification_report(test_labels, pred_labels))